In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing .image import ImageDataGenerator

In [ ]:
#define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [ ]:
#creating training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
)

In [ ]:
#creating training data with above parameters
#folder = parameters.flow_from_directory(path,ts,bs,cm,subset)
train_generator=train_datagen.flow_from_directory("/content/drive/MyDrive/Brain_Tumor_Detection-20240301T062122Z-001/Brain_Tumor_Detection/Train"
                                                  ,target_size=(IMG_SIZE,IMG_SIZE),
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='binary',
                                                 subset='training')

Found 2402 images belonging to 2 classes.


In [ ]:
#creating Validation data
val_generator = train_datagen.flow_from_directory("/content/drive/MyDrive/Brain_Tumor_Detection-20240301T062122Z-001/Brain_Tumor_Detection/Train"
                                                       ,target_size=(IMG_SIZE,IMG_SIZE),
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='binary',
                                                 subset='validation')

Found 600 images belonging to 2 classes.


In [ ]:
#define the model
model=keras.Sequential([
layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
layers.MaxPooling2D((2,2)),
layers.Conv2D(64,(3,3),activation='relu'),
layers.MaxPooling2D((2,2)),
layers.Conv2D(128,(3,3),activation='relu'),
layers.MaxPooling2D((2,2)),
layers.Flatten(),
layers.Dense(128,activation='relu'),
layers.Dense(1,activation='sigmoid')
])

In [ ]:
#compiling the model
#model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#compiling the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


76/76 [==============================] - 371s 5s/step - loss: 0.0000e+00 - accuracy: 0.5021 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/5
76/76 [==============================] - 309s 4s/step - loss: 0.0000e+00 - accuracy: 0.4996 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/5
76/76 [==============================] - 306s 4s/step - loss: 0.0000e+00 - accuracy: 0.4996 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 4/5
76/76 [==============================] - 282s 4s/step - loss: 0.0000e+00 - accuracy: 0.4996 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 5/5
76/76 [==============================] - 283s 4s/step - loss: 0.0000e+00 - accuracy: 0.4996 - val_loss: 0.0000e+00 - val_accuracy: 0.5000


In [ ]:
model.save("Model.h5","label.txt")

#RNN


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
import pandas as pd
df=pd.read_csv("/content/np.txt")
df

,Earth is the third planet from the Sun and the only astronomical object known to harbor life. This is enabled by Earth being a water world,the only one in the Solar System sustaining liquid surface water. Almost all of Earth's water is contained in its global ocean,covering 70.8% of Earth's crust. The remaining 29.2% of Earth's crust is land,most of which is located in the form of continental landmasses within one hemisphere,Earth's land hemisphere. Most of Earth's land is somewhat humid and covered by vegetation,while large sheets of ice at Earth's polar deserts retain more water than Earth's groundwater,lakes,rivers and atmospheric water combined. Earth's crust consists of slowly moving tectonic plates,which interact to produce mountain ranges,volcanoes,and earthquakes. Earth has a liquid outer core that generates a magnetosphere capable of deflecting most of the destructive solar winds and cosmic radiation.


In [ ]:
#Tokenizing the words
sentences = df
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
print(total_words)

95


In [ ]:
# Creating input sequences and their corresponding next words
input_sequences = []
for sentence in sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        n_gram_sequence = tokenized_sentence[:i+1]
        input_sequences.append(n_gram_sequence)
input_sequences

NameError: name 'sentences' is not defined

In [ ]:
# Padding sequences for consistent input size
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length,padding='pre')
print(input_sequences)

[[ 0  0  0 ...  0  7  4]
 [ 0  0  0 ...  7  4  2]
 [ 0  0  0 ...  4  2 21]
 ...
 [ 0  0  0 ... 92 17 93]
 [ 0  0  0 ... 17 93  5]
 [ 0  0  0 ... 93  5 94]]


In [ ]:
# Creating input and output data
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)


In [ ]:
# Building a simple RNN model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=50, input_length=max_sequence_length-1))
model.add(SimpleRNN(100, return_sequences=True))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(X,y,epochs=50,verbose=2)

NameError: name 'model' is not defined

In [ ]:
#Training the model
# Generating text using the trained model
seed_text = input("Enter the starting word: ")
next_words = int(input("Enter how many words to predict: "))

for _ in range(next_words):
    tokenized_seed = tokenizer.texts_to_sequences([seed_text])[0]
    tokenized_seed = pad_sequences([tokenized_seed], maxlen=max_sequence_length-1, padding='pre')
    predicted_word_index = np.argmax(model.predict(tokenized_seed), axis=-1)
    predicted_word = tokenizer.index_word[predicted_word_index[0]]
    seed_text += " " + predicted_word

print(seed_text)

Enter the starting word: earth
Enter how many words to predict: 10
1/1 [==============================] - 0s 14ms/step
earth land the third planet from the sun and the only
